In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV,  RandomizedSearchCV
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD


In [ ]:
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')  
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

To Add Two columns, One for a Year and the other one for the day of the Year

In [ ]:
def dayofyear(x):
    t = x.values
    y=[None]*len(t)
    P = [1,3, 5,7,8,10,12]
    I = [4,6,9,11]
    for i in range(len(t)):
        a = t[i]
        b = 0
        c= int(a[3:5])-1
        while c> 0 :
            if c in P:
                b =b+ 31
            if c in I:
                b=b+30
            if c ==2:
                b=b+28
            c = c -1
        y[i]=b+int(a[:2])
    return(pd.DataFrame(y))

In [ ]:
def theyear(x):
    t = x.values
    y=[None]*len(t)
    for i in range(0,len(t)):
        a = t[i]
        c= int(a[6:10])
        y[i] = c
    return(pd.DataFrame(y))

In [ ]:
train['Day'] = dayofyear(train['date'])
train['Year'] = theyear(train['date'])

In [ ]:
train.drop(('date'), axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
item_categories.head()

In [ ]:
items.head()

In [ ]:
shops.head()

In [ ]:
train.isnull().sum()

In [ ]:
# comparison of years over the different items

plt.rcParams['figure.figsize'] = (10, 5)
sns.countplot(train['Year'])
plt.title('Most sold items in every year', fontsize = 20)
plt.show()

In [ ]:
# comparison of years over the different items

plt.rcParams['figure.figsize'] = (10, 5)
sns.countplot(train['Year'])
plt.title('Most sold items in every year', fontsize = 20)
plt.show()

In [ ]:
# Comparing the items' price

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(train['item_price'], color = 'blue')
plt.xlabel('price Range for items', fontsize = 16)
plt.ylabel('Count of the item', fontsize = 16)
plt.title('Distribution of price of items', fontsize = 20)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# plotting a correlation heatmap

plt.rcParams['figure.figsize'] = (30, 20)
sns.heatmap(train.corr(), annot = True)

plt.title('Histogram of the Dataset', fontsize = 30)
plt.show()

In [ ]:
# finding 15 cheapest items from the dataset

train.sort_values('item_price', ascending = True)[['shop_id', 'item_id', 'item_price', 'item_cnt_day']].head(15).style.background_gradient('viridis')



In [ ]:
# finding 15 most expensive items from the dataset

train.sort_values('item_price', ascending = False)[['shop_id', 'item_id', 'item_price', 'item_cnt_day']].head(15).style.background_gradient('viridis')



In [ ]:
plt.scatter(x = 'shop_id', y = 'item_id', data = train)
plt.show()

In [ ]:
X = train[['shop_id','item_id']]
y = train[['item_cnt_day']]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)

In [ ]:
pca = PCA(n_components = 2)
X_train = pca.fit_transform(X_train)
X_valid = pca.transform(X_valid)

In [ ]:
train_mean_squared_error = {'Linear Regression':0, 'Polynomial Regression' :0,  'Decision Tree Regressor':0,
                            'XGBoost Regression':0, 'Random Forest Regressor':0}
test_mean_squared_error = {'Linear Regression':0, 'Polynomial Regression' :0,  'Decision Tree Regressor':0,
                            'XGBoost Regression':0, 'Random Forest Regressor':0}

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
train_preds = lr.predict(X_train)
test_preds = lr.predict(X_valid)
print("Linear Regression results :")
print("   -   The Mean Squared Error on the train set : {:.2f}".format(mean_squared_error(train_preds, y_train)))
train_mean_squared_error['Linear Regression'] = mean_squared_error(train_preds, y_train)
print("   -   The Mean Squared Error on the test set : {:.2f}".format(mean_squared_error(test_preds, y_valid)))
test_mean_squared_error['Linear Regression'] = mean_squared_error(test_preds, y_valid)


In [ ]:
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

In [ ]:
train_preds = lin_reg_2.predict(poly_reg.fit_transform(X_train))
test_preds = lin_reg_2.predict(poly_reg.fit_transform(X_valid))
print("Polynomiale Regression results :")
print("   -   The Mean Squared Error on the train set : {:.2f}".format(mean_squared_error(train_preds, y_train)))
train_mean_squared_error['Polynomial Regression'] = mean_squared_error(train_preds, y_train)
print("   -   The Mean Squared Error on the test set : {:.2f}".format(mean_squared_error(test_preds, y_valid)))
test_mean_squared_error['Polynomial Regression'] = mean_squared_error(test_preds, y_valid)

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

In [ ]:
train_preds = dt.predict(X_train)
test_preds = dt.predict(X_valid)
print("Decision Tree Regressor results :")
print("   -   The Mean Squared Error on the train set : {:.2f}".format(mean_squared_error(train_preds, y_train)))
train_mean_squared_error['Decision Tree Regressor'] = mean_squared_error(train_preds, y_train)
print("   -   The Mean Squared Error on the test set : {:.2f}".format(mean_squared_error(test_preds, y_valid)))
test_mean_squared_error['Decision Tree Regressor'] = mean_squared_error(test_preds, y_valid)

In [ ]:
GB = XGBRegressor()
GB.fit(X_train, y_train)

In [ ]:
train_preds = GB.predict(X_train)
test_preds = GB.predict(X_valid)
print("XGBoost Regression results :")
print("   -   The Mean Squared Error on the train set : {:.2f}".format(mean_squared_error(train_preds, y_train)))
train_mean_squared_error['XGBoost Regression'] = mean_squared_error(train_preds, y_train)
print("   -   The Mean Squared Error on the test set : {:.2f}".format(mean_squared_error(test_preds, y_valid)))
test_mean_squared_error['XGBoost Regression'] = mean_squared_error(test_preds, y_valid)

In [ ]:
rf = RandomForestRegressor(n_estimators = 10)
rf.fit(X_train, y_train)

In [ ]:
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_valid)
print("Random Forest Regressor results :")
print("   -   The Mean Squared Error on the train set : {:.2f}".format(mean_squared_error(train_preds, y_train)))
train_mean_squared_error['Random Forest Regressor'] = mean_squared_error(train_preds, y_train)
print("   -   The Mean Squared Error on the test set : {:.2f}".format(mean_squared_error(test_preds, y_valid)))
test_mean_squared_error['Random Forest Regressor'] = mean_squared_error(test_preds, y_valid)

In [ ]:
ind = np.arange(5)
width = 0.27

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)

rects1 = ax.bar(ind, list(train_mean_squared_error.values()), width, color='b')
rects2 = ax.bar(ind+width, list(test_mean_squared_error.values()), width, color='g')

ax.set_ylabel('Mean Squared Error ')
ax.set_xticks(ind+width)
ax.set_xticklabels( ('Linear Regression', 'Polynomial Regression', 'Decision Tree Regressor', 'XGBoost Regression', 'Random Forest Regressor') )
ax.legend((rects1[0], rects2[0]), ('Train MSER', 'Test MSER'))

plt.show()

In [ ]:
pred = lin_reg_2.predict(sc.transform(poly_reg.fit_transform(test[['shop_id','item_id']])))


In [ ]:
output = pd.DataFrame({'Id': test.ID ,'item_cnt_month': pred})

filename = 'Predictions1.csv'

output.to_csv(filename,index=False)

print('Saved file: ' + filename)